# Building a Chatbot

In this project, we will build a chatbot using conversations from Cornell University's [Movie Dialogue Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). The main features of our model are LSTM cells, a bidirectional dynamic RNN, and decoders with attention. 

The conversations will be cleaned rather extensively to help the model to produce better responses. As part of the cleaning process, punctuation will be removed, rare words will be replaced with "UNK" (our "unknown" token), longer sentences will not be used, and all letters will be in the lowercase. 

With a larger amount of data, it would be more practical to keep features, such as punctuation. However, I am using FloydHub's GPU services and I don't want to get carried away with too training for too long.

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time
from enum import Enum
tf.__version__

'1.7.0'

Most of the code to load the data is courtesy of https://github.com/suriyadeepan/practical_seq2seq/blob/master/datasets/cornell_corpus/data.py.

### Inspect and Load the Data

In [4]:
# Load the data
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [5]:
# The sentences that we will be using to train our model.
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [6]:
# The sentences' ids, which will be processed to become our input and target data.
conv_lines[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [7]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [8]:
# Create a list of all of the conversations' lines' ids.
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [9]:
convs[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

In [10]:
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

In [11]:
# Check if we have loaded the data correctly
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [12]:
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221616
221616


In [13]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [14]:
# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [15]:
# Take a look at some of the data to ensure that it has been cleaned well.
limit = 0
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [16]:
# Find the length of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [17]:
lengths.describe()

,counts
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [18]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
58.0


In [19]:
# Remove questions and answers that are shorter than 2 words and longer than 20 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

In [20]:
# Compare the number of lines we will use with the total number of lines.
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(questions),4)*100))

# of questions: 138335
# of answers: 138335
% of data used: 62.419999999999995%


In [21]:
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [22]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [23]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 45618
Size of vocab we will use: 8092


In [24]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.
questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

In [25]:
# Add the unique tokens to the vocabulary dictionaries.
pad = "<PAD>"
eos = "<EOS>"
unk = "<UNK>"
go = "<GO>"

codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)
    print(code, "<-->", questions_vocab_to_int[code]) #For debugging


<PAD> <--> 8092
<EOS> <--> 8093
<UNK> <--> 8094
<GO> <--> 8095


In [26]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [27]:
# Check the length of the dictionaries.
print(len(questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print(len(answers_int_to_vocab))

8096
8096
8096
8096


In [28]:
# Add the end of sentence token to the end of every answer.
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'

In [29]:
# Convert the text to integers. 
# Replace any words that are not in the respective vocabulary with <UNK> 
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [30]:
# Check the lengths
print(len(questions_int))
print(len(answers_int))

138335
138335


In [31]:
# Calculate what percentage of all words have been replaced with <UNK>
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 2334533
Number of times <UNK> is used: 92436
Percent of words that are <UNK>: 3.96%


In [32]:
# Sort questions and answers by the length of questions.
# This will reduce the amount of padding during training
# Which should speed up training and help to reduce the loss

sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))
print()
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print()

138335
138335

[58, 48]
[1, 59, 59, 59, 60, 61, 62, 1, 63, 12, 64, 36, 65, 66, 8093]

[1, 84]
[11, 192, 174, 55, 61, 21, 6, 8094, 159, 11, 8093]

[0, 85]
[153, 8, 9, 165, 11, 467, 55, 271, 8093]



In [33]:
#FIXME: This really should be something like "preprocess_targets"
def process_decoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)
    return dec_input


In [34]:
class GraphMode(Enum):
    TRAIN = 0
    VALID = 1
    TEST = 2
    SERVE = 3
    
#def convert_graph_mode(value, dtype=None, as_)

In [35]:
def dropout_cell(rnn_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    return tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)

def multi_dropout_cell(rnn_size, keep_prob, num_layers):    
    return tf.contrib.rnn.MultiRNNCell( [dropout_cell(rnn_size, keep_prob) for _ in range(num_layers)] )

In [36]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob): #, sequence_length):
    """
    Create the encoding layer
    
    Returns a tuple `(outputs, output_states)` where
      outputs is a 2-tuple of vectors of dimensions [sequence_length, rnn_size] for the forward and backward passes
      output_states is a 2-tupe of the final hidden states of the forward and backward passes
    
    """
    forward_cell = multi_dropout_cell(rnn_size, keep_prob, num_layers)
    backward_cell = multi_dropout_cell(rnn_size, keep_prob, num_layers)
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw = forward_cell,
                                                   cell_bw = backward_cell,
                                                   #sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return outputs, states

## Decoding

In [37]:
def decoding_layer(enc_state, enc_outputs, dec_embed_input, dec_embeddings, #Inputs
                        rnn_size, num_layers, output_layer, #Architecture
                        keep_prob, beam_width, #Hypeparameters
                        target_lengths, batch_size,
                        vocab_to_int): 
    
    with tf.variable_scope("decoding", reuse=tf.AUTO_REUSE) as decoding_scope:
        dec_cell = multi_dropout_cell(rnn_size, keep_prob, num_layers)
        init_dec_state_size = batch_size
        
        #TRAINING
        train_attn = tf.contrib.seq2seq.BahdanauAttention(num_units=dec_cell.output_size, memory=enc_outputs)
        train_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell, train_attn,
                                                    attention_layer_size=dec_cell.output_size)
        
        
        helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_lengths, time_major=False)
        train_decoder = tf.contrib.seq2seq.BasicDecoder(train_cell, helper,
                            train_cell.zero_state(init_dec_state_size, tf.float32).clone(cell_state=enc_state),
                            output_layer = output_layer)
        outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(train_decoder, scope=decoding_scope)
        logits = outputs[0]

        #INFERENCE
        #Tile inputs
        enc_state = tf.contrib.seq2seq.tile_batch(enc_state, beam_width)
        enc_outputs = tf.contrib.seq2seq.tile_batch(enc_outputs, beam_width)
        init_dec_state_size *= beam_width
        
        infer_attn = tf.contrib.seq2seq.BahdanauAttention(num_units=dec_cell.output_size, memory=enc_outputs)
        infer_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell, infer_attn,
                                                    attention_layer_size=dec_cell.output_size)
        
        
        decoder = tf.contrib.seq2seq.BeamSearchDecoder(cell = infer_cell,
            embedding = dec_embeddings,
            start_tokens = tf.tile([vocab_to_int["<GO>"]], [batch_size]), #Not by batch_size*beam_width, strangely
            end_token = vocab_to_int["<EOS>"],
            beam_width = beam_width,
            initial_state = infer_cell.zero_state(init_dec_state_size, tf.float32).clone(cell_state=enc_state),
            output_layer = output_layer
        )
            
        final_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, scope=decoding_scope)
        ids = final_decoder_output.predicted_ids
        beams = ids
                
    return logits, beams

In [38]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_lengths,
                  answers_vocab_size, questions_vocab_size, enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, beam_width, 
                  questions_vocab_to_int):
        
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       questions_vocab_size, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    
    enc_outputs, enc_states = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)    
    concatenated_enc_output = tf.concat(enc_outputs, -1)
    init_dec_state = enc_states[0]    
    
    dec_input = process_decoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([answers_vocab_size, dec_embedding_size], 0, 1))    
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    output_layer = tf.layers.Dense(answers_vocab_size)
    logits, beams = decoding_layer(init_dec_state,
                            concatenated_enc_output,
                            dec_embed_input,
                            dec_embeddings,
                            rnn_size, 
                            num_layers,
                            output_layer,
                            keep_prob,
                            beam_width,
                            target_sequence_lengths, 
                            batch_size,
                            answers_vocab_to_int,
                            )
    
    
    return logits, beams

In [39]:
# Set the Hyperparameters

#Network Architecture
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512

#Training
epochs = 100
batch_size = 128
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

#Decoding
beam_width = 10

In [40]:
def model_inputs(batch_size):
    '''Create palceholders for inputs to the model'''
    input_data = tf.placeholder(tf.int32, [batch_size, None], name='input')
    targets = tf.placeholder(tf.int32, [batch_size, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob

In [41]:
def pad_thru_time(predictions, targets, pad_index=answers_vocab_to_int[eos], vocab_size=len(answers_vocab_to_int)):
    """
    predictions - tensor of dimensions [batch_size, ?, vocab_size]
    targets - tensor of dimensions [batch_size, ?]
    pad_index - index of the token used to pad the shorter tensor
    vocab_size - size of the vocabulary
    
    Returns
       padded_pred, padded_targ
         which have equal lengths for their second axis
    """
    
    batch_size = predictions.shape[0]
    pred_length = predictions.shape[1]
    targ_length = targets.shape[1]
    
    max_length = tf.cond(tf.greater(pred_length, targ_length), lambda: pred_length, lambda: targ_length)
    print(max_length)
    
    pred_padding = tf.fill([batch_size, max_length - pred_length], pad_index)
    padded_pred = tf.concat([predictions, pred_padding], name="pad_predictions")
    print(padded_pred)
    
    pad_vector = tf.one_hot(pad_index, vocab_size)
    targ_padding = tf.tile( tf.tile(pad_vector, max_length - targ_length), batch_size )
    padded_targ = tf.concat([targets, targ_padding], axis=1, name="pad_targets")
    print(padded_targ)
    
    return padded_pred, padded_targ

In [45]:
def compute_lengths(generated, vocab_size, pad_id):
    """
    generated - [batch_size, max_pred_time]
    """
    time_axis = 1
    #one_cold - [batch_size, max_pred_time, vocab_size]
    one_cold = predictions = tf.one_hot(generated, vocab_size, on_value=0, off_value=1, axis = -1)
    lengths = tf.reduce_sum( one_cold[:, :, pad_id], axis = -1) #Counts non-padding tokens
    return lengths

In [ ]:
def pad_targets_body(i, accumulator, targets, pad_id, num_targ_pad):
    padded_vector = tf.concat( [targets[i], tf.tile(pad_id, num_targ_pad[i])], axis=-1 )
    #Add padded vector, increment i
    return tf.stack(accumulator, padded_vector), i + 1

def pad_generated_body(i, accumulator, predictions, pad_id, num_targ_pad, vocab_size):
    padded_vector = tf.concat([predictions[i], tf.tile(pad_id, num_targ_pad[i])], axis=-1)
    one_hot_padded = tf.one_hot(padded_vector, vocab_size, axis=-1)    
    #Add padded vector, increment i
    return tf.stack(accumulator, one_hot_padded), i + 1

def pad_predictions(predictions, targets, target_lengths, vocab_size, pad_id):
    """
    ids - tensor of dimensions [batch_size, max_pred_time]
    targets - tensor of dimensions [batch_size, max_target_time]
    """
    time_axis = 1
    batch_size = tf.shape(targets)[0]    
    
    max_pred_length = tf.shape(predictions)[time_axis]
    max_targ_length = tf.shape(targets)[time_axis]    
    max_length = tf.maximum(max_pred_length, max_targ_length)
    
    i0 = tf.constant(0)
    shape_invariants = [tf.get_shape(i0), tf.TensorShape(None, max_length, vocab_size)]
    accumulator = tf.zeros([1, max_length, vocab_size], tf.float32)
    pred_loop_variables = [i0, accumulator]
    num_pred_pad - max_length - max_targ_length
    padded_preds = tf.while(lambda i, accum: i < batch_size,
        lambda i, accum: pad_generated_body(i, accum, predictions, pad_id, num_pred_pad, vocab_size),
        pred_loop_variables,
        shape_invariants
    )[1: ] #Exclude entry 0, which is just a zero vector
    
    j0 = tf.constant(0)
    targ_accumulator = tf.zeros([1, max_length, vocab_size], tf.float32)
    targ_loop_variables = [j0, targ_accumulator]    
    targ_padding = max_length - max_targ_length
    padded_targs = tf.while(lambda j, accum: j < batch_size,
        lambda j, accum: pad_targets_body(j, accum, targets, pad_id, num_targ_pad),
        targ_loop_variables,
        shape_invariants #Use same shape invariants
    )[1:]
    
    return padded_preds, padded_targs

In [46]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()

    
# Placeholders for feed_dict    
input_data, targets, lr, keep_prob = model_inputs(batch_size)
input_shape = tf.shape(input_data)
#print("curr_batch_size =", curr_batch_size)

source_sequence_lengths = tf.placeholder(tf.int32, batch_size)
target_sequence_lengths = tf.placeholder(tf.int32, batch_size)

max_sequence_length_batch = tf.placeholder(tf.int32)

# Create the training and inference logits
#FIXME: Change "batch_size" to input_shape[0]?
train_logits, beams = \
seq2seq_model(tf.reverse(input_data, [-1]), targets, keep_prob, batch_size,# source_sequence_lengths,
    target_sequence_lengths, 
    len(answers_vocab_to_int), len(questions_vocab_to_int),
    encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, beam_width, questions_vocab_to_int)


with tf.name_scope("metrics"):
    # Find the shape of the input data for sequence_loss
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,
        tf.ones([batch_size, max_sequence_length_batch]) #FIXME: Weight the <PAD> tokens as 0
        )

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
    

    with tf.name_scope("evaluation"):
        prediction_ids = beams[:, :, 0] #Take best beam for each batch
        predictions = tf.one_hot(prediction_ids, len(answers_vocab_to_int), axis = -1)
        eos_vector = tf.one_hot(answers_vocab_to_int[eos], len(answers_vocab_to_int))
        print(predictions)
        print(eos_vector)
        
        true_lengths(predictions, answers_vocab_to_int[eos])
        
        #Ensure predicitons and targe
        #(padded_pred, padded_targ) = pad_thru_time(predictions, targets)

#         #Let's us ignore the extra padding when taking loss
#         eval_mask = tf.sequence_mask(target_sequence_lengths, maxlen=padded_pred.shape[1])
#         print(eval_mask)

#         eval_cost = tf.contrib.seq2seq.sequence_loss(
#             padded_pred,
#             padded_targets,
#             eval_mask
#         )


Tensor("metrics/evaluation/one_hot:0", shape=(128, ?, 8096), dtype=float32)
Tensor("metrics/evaluation/one_hot_1:0", shape=(8096,), dtype=float32)


ValueError: Shapes must be equal rank, but are 0 and 3 for 'metrics/evaluation/Select' (op: 'Select') with input shapes: [128,?,8096], [], [].

In [ ]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def batch_data(questions, answers, batch_size):
    """Batch questions and answers together"""
    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        
        true_q_lengths = np.asarray([len(sentence) for sentence in questions_batch], dtype=np.float32)
        true_a_lengths = np.asarray([len(sentence) for sentence  in answers_batch], dtype=np.float32)
        
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, questions_vocab_to_int))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, answers_vocab_to_int))
        
        yield pad_questions_batch, pad_answers_batch, true_q_lengths, true_a_lengths

In [ ]:
# Validate the training with 10% of the data
train_valid_split = int(len(sorted_questions)*0.15)

# Split the questions and answers into training and validating data
train_questions = sorted_questions[train_valid_split:]
train_answers = sorted_answers[train_valid_split:]

valid_questions = sorted_questions[:train_valid_split]
valid_answers = sorted_answers[:train_valid_split]

print(len(train_questions))
print(len(valid_questions))

In [ ]:
#TRAINING
display_step = 100 # Check training loss after every 100 batches
total_train_loss = 0 # Record the training loss for each display step

#VALIDATION
stop_early = 0 
stop = 5 # If the validation loss does decrease in 5 consecutive checks, stop training
validation_check = ((len(train_questions))//batch_size//2)-1 #Check validation loss every half-epoch
summary_valid_loss = [] # Record the validation loss for saving improvements in the model

checkpoint = "./checkpoints/best_model.ckpt" 

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(1, epochs+1):
        for batch_i, (questions_batch, answers_batch, q_lengths, a_lengths) in enumerate(
                batch_data(train_questions, train_answers, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: questions_batch,
                 targets: answers_batch,
                 lr: learning_rate,
                 source_sequence_lengths: q_lengths,
                 target_sequence_lengths: a_lengths,
                 max_sequence_length_batch: answers_batch.shape[1],
                 keep_prob: keep_probability})

            total_train_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time
            
            total_train_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_questions) // batch_size, 
                              total_train_loss / display_step, 
                              batch_time*display_step))
                total_train_loss = 0

            if batch_i % validation_check == 0 and batch_i > 0:
                total_valid_loss = 0
                start_time = time.time()
                for batch_ii, (questions_batch, answers_batch) in \
                        enumerate(batch_data(valid_questions, valid_answers, batch_size)):
                    valid_loss = sess.run(
                    cost, {input_data: questions_batch,
                           targets: answers_batch,
                           lr: learning_rate,
                           max_sequence_length_batch: answers_batch.shape[1],
                           keep_prob: 1})
                    total_valid_loss += valid_loss
                end_time = time.time()
                batch_time = end_time - start_time
                avg_valid_loss = total_valid_loss / (len(valid_questions) / batch_size)
                print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))

                # Reduce learning rate, but not below its minimum value
                learning_rate *= learning_rate_decay
                if learning_rate < min_learning_rate:
                    learning_rate = min_learning_rate

                summary_valid_loss.append(avg_valid_loss)
                if avg_valid_loss <= min(summary_valid_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
    
        if stop_early == stop:
            print("Stopping Training.")
            break


In [ ]:
def question_to_seq(question, vocab_to_int):
    '''Prepare the question for the model'''
    
    question = clean_text(question)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in question.split()]

In [ ]:
# Create your own input question
#input_question = 'How are you?'

# Use a question from the data as your input
random = np.random.choice(len(short_questions))
input_question = short_questions[random]

# Prepare the question
input_question = question_to_seq(input_question, questions_vocab_to_int)

# Pad the questions until it equals the max_line_length
input_question = input_question + [questions_vocab_to_int["<PAD>"]] * (max_line_length - len(input_question))
# Add empty questions so the the input_data is the correct shape
batch_shell = np.zeros((batch_size, max_line_length))
# Set the first question to be out input question
batch_shell[0] = input_question    

saver = tf.train.Saver()
with tf.Session() as sess:
    # Run the model with the input question
    saver.restore(sess, checkpoint)
    beams = sess.run(infer_ids, {input_data: batch_shell, 
                                                keep_prob: 1.0})[0]
# Remove the padding from the Question and Answer
pad_q = questions_vocab_to_int["<PAD>"]
pad_a = answers_vocab_to_int["<EOS>"]    
print('Question')
print('  Word Ids:      {}'.format([i for i in input_question if i != pad_q]))
print('  Input Words: {}'.format([questions_int_to_vocab[i] for i in input_question if i != pad_q]))
print(beams)

for i in range(beam_width):
    beam = beams[:, i]
    print('\nAnswer', i)
    print('  Word Ids:      {}'.format([i for i in beam if i != pad_a]))
    print('  Response Words: {}'.format([answers_int_to_vocab[i] for i in beam if i != pad_a]))